In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.ensemble import AdaBoostClassifier

In [8]:
data_pd = pd.read_csv('titanic_preprocessed.csv')
data_pd = data_pd.sort_values(by = 'PassengerId')
test_pd = data_pd[data_pd['Survived'].isna()]
test_pd['PassengerId'] = test_pd['PassengerId'] + 1
test_pd_tmp = test_pd.drop(columns = ['Survived','PassengerId'])
display(data_pd)
display(test_pd)

/tmp/ipykernel_712188/1356244638.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_pd['PassengerId'] = test_pd['PassengerId'] + 1


,PassengerId,Pclass,SibSp,Parch,Fare_Bin,Num_family,Age_Bin,Sex_0,Sex_1,Embarked_0,Embarked_1,Embarked_2,Title_0,Title_1,Title_2,Title_3,Title_4,Title_5,Title_6,Survived
263,0,2,1,0,0,2,2,False,True,False,False,True,False,False,True,False,False,False,False,0.0
264,1,0,1,0,1,2,3,True,False,True,False,False,False,False,False,True,False,False,False,1.0
265,2,2,0,0,0,0,2,True,False,False,False,True,False,False,False,False,True,False,False,1.0
266,3,0,1,0,1,2,3,True,False,False,False,True,False,False,False,True,False,False,False,1.0
267,4,2,0,0,0,0,3,False,True,False,False,True,False,False,True,False,False,False,False,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,1304,2,0,0,0,0,2,False,True,False,False,True,False,False,True,False,False,False,False,NaN
1307,1305,0,0,0,1,0,3,True,False,True,False,False,False,False,False,False,False,True,False,NaN
1308,1306,2,0,0,0,0,3,False,True,False,False,True,False,False,True,False,False,False,False,NaN
261,1307,2,0,0,0,0,2,False,True,False,False,True,False,False,True,False,False,False,False,NaN


,PassengerId,Pclass,SibSp,Parch,Fare_Bin,Num_family,Age_Bin,Sex_0,Sex_1,Embarked_0,Embarked_1,Embarked_2,Title_0,Title_1,Title_2,Title_3,Title_4,Title_5,Title_6,Survived
977,892,2,0,0,0,0,3,False,True,False,True,False,False,False,True,False,False,False,False,NaN
978,893,2,1,0,0,2,4,True,False,False,False,True,False,False,False,True,False,False,False,NaN
979,894,1,0,0,0,0,6,False,True,False,True,False,False,False,True,False,False,False,False,NaN
980,895,2,0,0,0,0,2,False,True,False,False,True,False,False,True,False,False,False,False,NaN
981,896,2,1,1,0,3,2,True,False,False,False,True,False,False,False,True,False,False,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,1305,2,0,0,0,0,2,False,True,False,False,True,False,False,True,False,False,False,False,NaN
1307,1306,0,0,0,1,0,3,True,False,True,False,False,False,False,False,False,False,True,False,NaN
1308,1307,2,0,0,0,0,3,False,True,False,False,True,False,False,True,False,False,False,False,NaN
261,1308,2,0,0,0,0,2,False,True,False,False,True,False,False,True,False,False,False,False,NaN


In [9]:
# data_pd = pd.read_csv('titanic_preprocessed.csv')
# data_pd.set_index("index",inplace=True)
data_pd = data_pd.dropna(subset=['Survived'])
X_pd = data_pd.drop(columns = ['Survived','PassengerId'])
y_pd = data_pd['Survived']
X_pd,y_pd

(     Pclass  SibSp  Parch  Fare_Bin  Num_family  Age_Bin  Sex_0  Sex_1  \
 263       2      1      0         0           2        2  False   True   
 264       0      1      0         1           2        3   True  False   
 265       2      0      0         0           0        2   True  False   
 266       0      1      0         1           2        3   True  False   
 267       2      0      0         0           0        3  False   True   
 ..      ...    ...    ...       ...         ...      ...    ...    ...   
 973       1      0      0         0           0        2  False   True   
 974       0      0      0         1           0        1   True  False   
 176       2      1      2         0           4        0   True  False   
 975       0      0      0         1           0        2  False   True   
 976       2      0      0         0           0        3  False   True   
 
      Embarked_0  Embarked_1  Embarked_2  Title_0  Title_1  Title_2  Title_3  \
 263       False  

split the data

In [10]:

# split
X = X_pd.values
y = y_pd.values

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

print(len(X_scaled))
print(len(y))

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state= 42)



891
891


RandomForest

In [11]:

# initialize RandomForestClassifier
rf_clf = RandomForestClassifier()

rf_param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5],
    'min_samples_split': [2, 4]
}

rf_grid_search = GridSearchCV(rf_clf, rf_param_grid, cv=3, scoring='f1')

start_time = time.time()
rf_grid_search.fit(X_train, y_train)
rf_time = time.time() - start_time

rf_best = rf_grid_search.best_estimator_

# 在测试集上进行预测
rf_y_pred = rf_best.predict(X_test)

rf_accuracy = accuracy_score(y_test, rf_y_pred)
rf_f1 = f1_score(y_test, rf_y_pred)

print(f"Random Forest best parameters: {rf_grid_search.best_params_}")
print(f"Random Forest best F1 score: {rf_grid_search.best_score_}")
print(f"Random Forest accuracy: {rf_accuracy}")
print(f"Random Forest run time: {rf_time}")

Random Forest best parameters: {'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 200}
Random Forest best F1 score: 0.7677871148459383
Random Forest accuracy: 0.8156424581005587
Random Forest run time: 1.2801342010498047


XGBoost

In [15]:
xgb_clf = xgb.XGBClassifier()

xgb_param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.5],
    'max_depth': [3, 4, 5]
}

xgb_grid_search = GridSearchCV(xgb_clf, xgb_param_grid, cv=3, scoring='f1')

start_time = time.time()
xgb_grid_search.fit(X_train, y_train)
xgb_time = time.time() - start_time

xgb_best = xgb_grid_search.best_estimator_

xgb_y_pred = xgb_best.predict(X_test)

xgb_accuracy = accuracy_score(y_test, xgb_y_pred)
xgb_f1 = f1_score(y_test, xgb_y_pred)

print(f"XGBoost best parameters: {xgb_grid_search.best_params_}")
print(f"XGBoost best F1 score: {xgb_grid_search.best_score_}")
print(f"XGBoost accuracy: {xgb_accuracy}")
print(f"XGBoost run time: {xgb_time}")

XGBoost best parameters: {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100}
XGBoost best F1 score: 0.7531441717791411
XGBoost accuracy: 0.8324022346368715
XGBoost run time: 8.453047752380371


AdamBoost

In [16]:

ada_clf = AdaBoostClassifier(n_estimators=100, random_state=42)

ada_param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 1.0]
}

ada_grid_search = GridSearchCV(estimator=ada_clf, param_grid=ada_param_grid, cv=5, scoring='accuracy')

start_time = time.time()
ada_grid_search.fit(X_train, y_train)
ada_time = time.time() - start_time

#find the best model
ada_best = ada_grid_search.best_estimator_

# predict with the best model
ada_y_pred = ada_best.predict(X_test)

ada_accuracy = accuracy_score(y_test, ada_y_pred)
ada_f1 = f1_score(y_test, xgb_y_pred)

print(f"AdaBoost best parameters: {ada_grid_search.best_params_}")
print(f"AdaBoost best F1 score: {ada_grid_search.best_score_}")
print(f"AdaBoost accuracy: {ada_accuracy}")
print(f"AdaBoost run time: {ada_time}")

/usr/local/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.1

AdaBoost best parameters: {'learning_rate': 0.1, 'n_estimators': 100}
AdaBoost best F1 score: 0.8342361863488623
AdaBoost accuracy: 0.8156424581005587
AdaBoost run time: 2.4843029975891113


/usr/local/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


In [ ]:
X_test_scaled = scaler.transform(test_pd_tmp)

# use random forest model to predict the suvivors in test.csv
test_predictions = random_forest_model.predict(X_test_scaled)

len(test_predictions)
# print(test_predictions)

result_randomforest = pd.DataFrame({'PassengerId': test_pd['PassengerId'], 'Survived': test_predictions.astype(int)})

result_randomforest.to_csv('randomforest_predictions.csv', index=False)

/usr/local/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


In [ ]:
X_xg_test_scaled = scaler.transform(test_pd_tmp)

# use random forest model to predict the suvivors in test.csv
XG_test_predictions = xgb_clf.predict(X_xg_test_scaled)

len(test_predictions)
# print(test_predictions)

result_randomforest = pd.DataFrame({'PassengerId': test_pd['PassengerId'], 'Survived': XG_test_predictions.astype(int)})

result_randomforest.to_csv('xgboost_predictions.csv', index=False)

/usr/local/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
